In [4]:
import requests
import ipywidgets as widgets
import IPython.display

In [9]:
url = 'http://127.0.0.1:5000/test'

data_box = widgets.Text(
    value='mydata',
    placeholder='Type something',
    description='Data:',
    disabled=False
)
button = widgets.Button(description="POST data")
output = widgets.Output()

display(data_box, button, output)

def on_post_button_clicked(b):
    with output:
        x = requests.post(url, data = data_box.value)
        display(x.text)

button.on_click(on_post_button_clicked)

Text(value='mydata', description='Data:', placeholder='Type something')

Button(description='POST data', style=ButtonStyle())

Output()